In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/marketing-strategy-personalised-offer/sample.csv
/kaggle/input/marketing-strategy-personalised-offer/train_data.csv
/kaggle/input/marketing-strategy-personalised-offer/test_data.csv
/kaggle/input/dataset/sample.csv
/kaggle/input/dataset/train_data.csv
/kaggle/input/dataset/test_data.csv


# **Libraries**

In [2]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

# **Load the data**

In [3]:
#Loading the data
data_train=pd.read_csv('../input/dataset/train_data.csv')
data_test=pd.read_csv('../input/dataset/test_data.csv')

# **Analyzing the Data**

In [4]:
#data_train.info()
# data_train.isnull().sum()

In [5]:
data_train=data_train.drop('car',axis='columns')
data_test=data_test.drop('car', axis='columns')

In [6]:
X_test=data_test

In [7]:
y_train=data_train[data_train.columns[29]]
X_train=data_train[data_train.columns[:29]]

In [8]:
X_train.head()

,offer expiration,income_range,no_visited_Cold drinks,travelled_more_than_15mins_for_offer,Restaur_spend_less_than20,Marital Status,restaurant type,age,Prefer western over chinese,travelled_more_than_25mins_for_offer,...,Job/Job Industry,restuarant_opposite_direction_house,has Children,visit restaurant with rating (avg),temperature,Restaur_spend_greater_than20,Travel Time,Climate,drop location,Prefer home food
0,2days,₹100000 or More,4~8,1,less1,Married partner,4 star restaurant,36,0,0,...,Unemployed,0,0,4,67,less1,22,Spring,Location B,0
1,2days,₹87500 - ₹99999,4~8,0,4~8,Married partner,Take-away restaurant,50plus,0,0,...,Unemployed,0,1,3,89,1~3,18,Summer,Location B,0
2,2days,₹87500 - ₹99999,less1,1,1~3,Single,Cold drinks,26,1,0,...,Arts Design Entertainment Sports & Media,1,1,4,67,less1,7,Winter,Location A,1
3,10hours,₹37500 - ₹49999,less1,0,1~3,Single,Take-away restaurant,46,1,0,...,Sales & Related,0,1,3,89,1~3,7,Summer,Location C,0
4,2days,₹100000 or More,never,1,1~3,Single,4 star restaurant,21,0,1,...,Sales & Related,1,0,3,40,less1,7,Summer,Location C,0


# **Preprocessing the data**

In [9]:
#data_train['gender'].unique()

In [10]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# cor_matrix=data_train.corr()
# plt.figure(figsize=(40,40))
# sns.heatmap(cor_matrix,annot=True)

In [11]:
X_train=pd.get_dummies(X_train, columns=['Marital Status', 'restaurant type', 'gender', 'Customer type', 'Climate', 'drop location', 'Job/Job Industry'], prefix='onehot')
X_test=pd.get_dummies(X_test, columns=['Marital Status', 'restaurant type', 'gender', 'Customer type', 'Climate', 'drop location', 'Job/Job Industry'], prefix='onehot')
#X_train=pd.get_dummies(X_train, columns=['Marital Status', 'restaurant type', 'gender', 'Customer type', 'Qualification', 'Climate', 'drop location'], prefix='onehot')

In [12]:
income_range_categories=[np.nan, 'Less than ₹12500', '₹12500 - ₹24999', '₹25000 - ₹37499', '₹37500 - ₹49999', '₹50000 - ₹62499', '₹62500 - ₹74999', '₹75000 - ₹87499', '₹87500 - ₹99999', '₹100000 or More']
offer_exp_categories=[np.nan, '10hours', '2days']
visited_Cold_drinks=[np.nan, 'never', 'less1', '1~3', '4~8', 'gt8']
#Restaur_spend_less_than20_categories=['never', 'less1', '1~3', '4~8', 'gt8']
age_categories=[np.nan, 'below21', '21', '26', '31', '36', '41', '46', '50plus']
no_visited_bars_categories=[np.nan, 'never', 'less1', '1~3', '4~8', 'gt8']
Qualification_categories=[np.nan, 'Some High School', 'High School Graduate', 'Some college - no degree', 'Associates degree', 'Bachelors degree', 'Graduate degree (Masters or Doctorate)']
#no_Take-aways_categories=['never', 'less1', '1~3', '4~8', 'gt8']
#Restaur_spend_greater_than20_categories=['never', 'less1', '1~3', '4~8', 'gt8']

In [13]:
oe1=OrdinalEncoder(categories=[income_range_categories])
X_train['income_range'] = oe1.fit_transform(X_train.income_range.values.reshape(-1,1))
X_test['income_range'] = oe1.fit_transform(X_test.income_range.values.reshape(-1,1))

In [14]:
# oe2=OrdinalEncoder(categories=[offer_exp_categories])
# X_train['offer expiration'] = oe2.fit_transform(X_train['offer expiration'].values.reshape(-1,1))
# X_test['offer expiration'] = oe2.fit_transform(X_test['offer expiration'].values.reshape(-1,1))

In [15]:
X_train['offer expiration']=X_train['offer expiration'].replace('2days', 48)
X_train['offer expiration']=X_train['offer expiration'].replace('10hours', 10)
X_test['offer expiration']=X_test['offer expiration'].replace('2days', 48)
X_test['offer expiration']=X_test['offer expiration'].replace('10hours', 10)

In [16]:
oe3=OrdinalEncoder(categories=[visited_Cold_drinks])
X_train['no_visited_Cold drinks'] = oe3.fit_transform(X_train['no_visited_Cold drinks'].values.reshape(-1,1))
X_test['no_visited_Cold drinks'] = oe3.fit_transform(X_test['no_visited_Cold drinks'].values.reshape(-1,1))

In [17]:
oe4=OrdinalEncoder(categories=[visited_Cold_drinks])
X_train['Restaur_spend_less_than20'] = oe4.fit_transform(X_train['Restaur_spend_less_than20'].values.reshape(-1,1))
X_test['Restaur_spend_less_than20'] = oe4.fit_transform(X_test['Restaur_spend_less_than20'].values.reshape(-1,1))

In [18]:
oe5=OrdinalEncoder(categories=[age_categories])
X_train['age'] = oe5.fit_transform(X_train['age'].values.reshape(-1,1))
X_test['age'] = oe5.fit_transform(X_test['age'].values.reshape(-1,1))

In [19]:
oe6=OrdinalEncoder(categories=[no_visited_bars_categories])
X_train['no_visited_bars'] = oe6.fit_transform(X_train['no_visited_bars'].values.reshape(-1,1))
X_test['no_visited_bars'] = oe6.fit_transform(X_test['no_visited_bars'].values.reshape(-1,1))

In [20]:
oe7=OrdinalEncoder(categories=[Qualification_categories])
X_train['Qualification'] = oe7.fit_transform(X_train['Qualification'].values.reshape(-1,1))
X_test['Qualification'] = oe7.fit_transform(X_test['Qualification'].values.reshape(-1,1))

In [21]:
oe8=OrdinalEncoder(categories=[visited_Cold_drinks])
X_train['no_Take-aways'] = oe8.fit_transform(X_train['no_Take-aways'].values.reshape(-1,1))
X_test['no_Take-aways'] = oe8.fit_transform(X_test['no_Take-aways'].values.reshape(-1,1))

In [22]:
oe9=OrdinalEncoder(categories=[visited_Cold_drinks])
X_train['Restaur_spend_greater_than20'] = oe9.fit_transform(X_train['Restaur_spend_greater_than20'].values.reshape(-1,1))
X_test['Restaur_spend_greater_than20'] = oe9.fit_transform(X_test['Restaur_spend_greater_than20'].values.reshape(-1,1))

In [23]:
X_train.head()

,offer expiration,income_range,no_visited_Cold drinks,travelled_more_than_15mins_for_offer,Restaur_spend_less_than20,age,Prefer western over chinese,travelled_more_than_25mins_for_offer,travelled_more_than_5mins_for_offer,no_visited_bars,...,onehot_Management,onehot_Office & Administrative Support,onehot_Personal Care & Service,onehot_Production Occupations,onehot_Protective Service,onehot_Retired,onehot_Sales & Related,onehot_Student,onehot_Transportation & Material Moving,onehot_Unemployed
0,48,9.0,4.0,1,2.0,5.0,0,0,1,2.0,...,0,0,0,0,0,0,0,0,0,1
1,48,8.0,4.0,0,4.0,8.0,0,0,1,1.0,...,0,0,0,0,0,0,0,0,0,1
2,48,8.0,2.0,1,3.0,3.0,1,0,1,1.0,...,0,0,0,0,0,0,0,0,0,0
3,10,4.0,2.0,0,3.0,7.0,1,0,1,1.0,...,0,0,0,0,0,0,1,0,0,0
4,48,9.0,1.0,1,3.0,2.0,0,1,1,2.0,...,0,0,0,0,0,0,1,0,0,0


In [24]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)

In [25]:
y_train

array([0, 1, 1, ..., 0, 1, 1])

In [26]:
X_train_copy = X_train
y_train_copy = y_train

In [27]:
imputer = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
X_train_copy = imputer.fit_transform(X_train_copy)
X_test = imputer.fit_transform(X_test)

# Model Training

In [28]:
from sklearn.model_selection import train_test_split
x_train, x_test, Y_train, Y_test = train_test_split(X_train_copy, y_train, test_size=0.25, random_state=32)

In [29]:
#from sklearn.dummy import DummyClassifier
# dummy_clf = DummyClassifier()
# dummy_clf.fit(x_train, Y_train)
# y_pred_dc = dummy_clf.predict(x_test)
# dc_sc=f1_score(Y_test, y_pred_dc)
#dc_sc
#0.721602643535729
#0.5583

In [30]:
#from sklearn.linear_model import Perceptron
# bin_clf = Perceptron(max_iter=100, random_state=42)
# bin_clf.fit(x_train, Y_train)
# y_pred_bin_clf = bin_clf.predict(x_test)
# bin_clf_sc=f1_score(Y_test, y_pred_bin_clf)
# bin_clf_sc
#0.6096813725490197
#0.57608

In [31]:
#from sklearn.linear_model import SGDClassifier
# sgd_clf = SGDClassifier(loss="huber", alpha=0.1)
# sgd_clf.fit(x_train, Y_train)
# y_pred_sgd=sgd_clf.predict(x_test)
# sgd_sc=f1_score(Y_test, y_pred_sgd)
# sgd_sc
#0.7106852739012706
#0.54349 

In [32]:
# from sklearn.linear_model import RidgeClassifier
# r_clf=RidgeClassifier(fit_intercept=False, random_state=42)
# r_clf.fit(x_train, Y_train)
# y_pred_rclf=r_clf.predict(x_test)
# r_clf_sc=f1_score(Y_test, y_pred_rclf)
# r_clf_sc
#0.6862745098039217

In [33]:
# from sklearn.linear_model import LogisticRegression
# logit = LogisticRegression(random_state=42)
# logit.fit(x_train,Y_train)
# y_pred_logit = logit.predict(x_test)
# logit_sc=f1_score(Y_test, y_pred_logit)
# logit_sc
#0.688659793814433

In [34]:
#from sklearn.linear_model import LogisticRegressionCV
# logit_cv = LogisticRegressionCV(cv=5, scoring='f1',random_state=42)
# logit_cv.fit(x_train,Y_train)
# y_pred_logit_cv = logit_cv.predict(x_test)
# logit_cv_sc=f1_score(Y_test, y_pred_logit_cv)
#0.7041214750542301
#0.56854

In [35]:
# from sklearn.neighbors import KNeighborsClassifier
# knn=KNeighborsClassifier(n_neighbors=35)
# knn.fit(x_train,Y_train)
# knn_sc  = f1_score(Y_test, knn.predict(x_test))
# knn_sc
#0.55696
#0.6658409710180828

In [36]:
# from sklearn.tree import DecisionTreeClassifier
# dtc=DecisionTreeClassifier(max_depth=5)
# dtc.fit(x_train, Y_train)
# dtc_pred=dtc.predict(x_test)
# dtc_pred
# sc = dtc.score(x_test, Y_test)
# print(sc)
# dtc.get_n_leaves()


In [37]:
# from sklearn.tree import plot_tree
# plot_tree(dtc)

In [38]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth=15, random_state=10, n_estimators=550)
rfc.fit(x_train, Y_train)
y_pred_rfc = rfc.predict(x_test)
sc = f1_score(Y_test, y_pred_rfc)


In [39]:
sc

0.7146426786606697

In [40]:
# from sklearn.neural_network import MLPClassifier
# clf=MLPClassifier(hidden_layer_sizes=(140,), random_state=10, max_iter=1000)
# clf.fit(x_train, Y_train)
# y_pred_clf=clf.predict(x_test)
# scor=f1_score(Y_test, y_pred_clf)
# scor
# # 0.7012195121951219

In [41]:
from sklearn.ensemble import AdaBoostClassifier
abc=AdaBoostClassifier(base_estimator=rfc, random_state=10)
abc.fit(x_train, Y_train)
y_pred_abc=abc.predict(x_test)
sco=f1_score(Y_test, y_pred_abc)


In [42]:
sco

0.7139611209290584

In [43]:
#?AdaBoostClassifier

In [44]:
# from sklearn.ensemble import GradientBoostingClassifier
# gbc=GradientBoostingClassifier()
# gbc.fit(x_train, Y_train)
# y_pred_gbc=gbc.predict(x_test)
# sco=f1_score(Y_test, y_pred_abc)
# sco

In [45]:
# param_grid={'loss'=[‘log_loss’, ‘deviance’, ‘exponential’], criterion=[‘friedman_mse’, ‘squared_error’]}

In [46]:
# gv=GridSearchCV(gbc, param_grid=param_grid)
# gv.fit(X_train_copy, y_train)
# gv.best_params

In [47]:
Y_p=abc.predict(X_test)
Y_p.shape

(5305,)

In [48]:
y_pred=le.inverse_transform(Y_p)
pred_id = range(X_test.shape[0])
sub = pd.DataFrame(data={'id':pred_id, 'Offer Accepted':y_pred})

In [49]:
#sub

In [50]:
sub.to_csv('submission.csv',index=False)